In [0]:
%pip install mlflow xgboost

%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0

%restart_python

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import DataFrame, functions as F, types as T, Window

import builtins
from datetime import datetime
from typing import Optional, Dict, Union, List, Tuple, Any
import math
import random


import pandas as pd
import numpy as np
import sklearn

from xgboost.spark import SparkXGBClassifier, SparkXGBRegressor
import mlflow

from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics


from pyspark.ml.feature import BucketedRandomProjectionLSH
from pyspark.ml.linalg import Vectors, DenseVector, SparseVector, VectorUDT
from pyspark.ml import Pipeline, PipelineModel


from pyspark.ml.tuning import CrossValidatorModel, TrainValidationSplitModel, ParamGridBuilder, CrossValidator, TrainValidationSplit
from pyspark.storagelevel import StorageLevel

import matplotlib.pyplot as plt

from pyspark.sql.functions import round
import mlflow.spark
from mlflow.artifacts import download_artifacts
from matplotlib import colormaps

In [0]:
#model_uri = 'runs:/170ea1a3b25b4540b9138b6abd731b44/best_model/spark-model'
#model = mlflow.pyfunc.load_model(model_uri)

In [0]:
#NP - churn label 7 - stratified
run_id = 'f5583d4499394f16aa8b35757ad46e81'
model_uri = f"runs:/{run_id}/best_model/spark-model"

In [0]:
model = mlflow.spark.load_model(model_uri)

In [0]:
model.stages[-2].getInputCols()

In [0]:

df = pd.DataFrame(list(zip(model.stages[-2].getInputCols(),model.stages[-1].get_feature_importances().values())), columns = ['features','feature_importances']).set_index('features').sort_values('feature_importances', ascending=False).head(20)
#import seaborn as sns


In [0]:
df

In [0]:
df

In [0]:
mlflow.artifacts.list_artifacts("dbfs:/databricks/mlflow-tracking/762753878692720/00d93806149f4fdfbc3eed66e416c8d6/artifacts/plots")

In [0]:
mlflow.artifacts.list_artifacts("dbfs:/databricks/mlflow-tracking/762753878692720/00d93806149f4fdfbc3eed66e416c8d6/artifacts/plots")

In [0]:
model_uri

In [0]:
#TODO: Log things like p/r points over thresholds

In [0]:
from mlflow import MlflowClient

client = MlflowClient()
run = client.get_run('170ea1a3b25b4540b9138b6abd731b44')


run.data.metrics


In [0]:
def _get_feature_importances(best, top_n = 20):
    feature_names = best.stages[-2].getInputCols()
    feature_importances = best.stages[-1].get_feature_importances().values()

    df = pd.DataFrame(list(zip(feature_names, feature_importances)), columns = ['features','feature_importances']).set_index('features').sort_values('feature_importances', ascending=False).head(top_n)

    return df 
    
def _plt_feature_importances(df):

    from matplotlib import colormaps
    my_cmap = plt.get_cmap("viridis")

    rescale = lambda y: (y - np.min(y)) / (np.max(y) - np.min(y))

    fig, ax = plt.subplots()
    plt.barh(df.index, df['feature_importances'], color=my_cmap(rescale(df['feature_importances'])))
    ax.set_title("Feature importances using MDI")
    ax.set_ylabel("Mean decrease in impurity")
    fig.tight_layout()

    return fig

In [0]:
features

In [0]:
features = _get_feature_importances(model,20)

feature_plot = _plt_feature_importances(features)
#feature_plot

In [0]:
my_cmap = plt.get_cmap("viridis")
rescale = lambda y: (y - np.min(y)) / (np.max(y) - np.min(y))

plt.bar(df.index, df['feature_importances'], color=my_cmap(rescale(df['feature_importances'])))

In [0]:
plt.barh(df.index, df['feature_importances'], color=my_cmap(rescale(df['feature_importances'])))

In [0]:

#sns.barplot(x='importance', y='feature', data=feature_importance_df)
#sers = pd.Series(df['feature_importances'], index=df['features']).sort_values(ascending=False)

my_cmap = plt.get_cmap("viridis")
fig, ax = plt.subplots()
df['feature_importances'].plot.barh(ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()


#sers.plot.barh(figsize=(10,10))

In [0]:
model.stages[-1].get_feature_importances()

In [0]:
#model.unwrap_python_model()

In [0]:
loaded_model = mlflow.pyfunc.spark_udf(spark, model_uri=model_uri)

# Predict on a Spark DataFrame.